In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
from numpy import radians, sin, cos, arcsin, sqrt
def dist(a, b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    x1, y1, x2, y2 = map(radians, [a[0], a[1], b[0], b[1]])
    dx = x2 - x1
    dy = y2 - y1
    a = sin(dy/2)**2 + cos(y1) * cos(y2) * sin(dx/2)**2
    c = 2 * arcsin(sqrt(a))
    # Earth`s radius in kilometers
    km = 6372.795 * c
    return km

In [ ]:
plt.style.use('classic')   # try another styles: 'classic'

srcData = "G:/1_Data1/sea/AllSea.txt"
destData = "G:/1_Data1/sea/AllSeaGisOut.txt"
obrisData = "G:/1_Data1/obris/obris.dat"
# srcData = "~/methaneSea.dat

In [ ]:
df = pd.read_csv(srcData,
                 header=0,
                 # skip_blank_lines=True,
                 skipinitialspace=True,
                 na_values='--',
                 skiprows=[1,2,3,4],
                 sep='\t',
                 decimal=',',
                 # nrows=100000,
                 parse_dates=['DateTime'],
                 dayfirst=True
                 )
df.dtype

In [ ]:
obris = pd.read_csv(obrisData,
                    header=0,
                    na_values='--',
                    sep='\t',
                    decimal=',',
                    )

In [ ]:
longStep = 0.01
latStep = 0.005

df['long'] = (df['Longitude']/longStep).round()*longStep
df['lat'] = (df['Latitude']/latStep).round()*latStep
df['year'] = df['DateTime'].dt.year

In [ ]:
cols_to_calc = {'pCO2Wtr': ['mean', 'std', 'count'],
                'pCH4Wtr': ['mean', 'std', 'count'],
                'cCO2Wtr': ['mean', 'std', 'count'],
                'cCH4Wtr': ['mean', 'std', 'count'],
                'dpCO2': ['mean', 'std', 'count'],
                'dpCH4': ['mean', 'std', 'count'],
                'pCO2Air': ['mean', 'std', 'count'],
                'pCH4Air': ['mean', 'std', 'count'],}
res = df.groupby(['year', 'long', 'lat'], as_index=False).agg(cols_to_calc)
res

In [ ]:
res['toobris'] = 0

for i in range(len(res)):
    min_dist = 100
    for j in range(len(obris)):
        dista = dist((res['long'][i], res['lat'][i]), (obris['long'][j], obris['lat'][j]))
        if (min_dist > dista):
            min_dist = dista
    res.at[i, 'toobris'] = min_dist

In [ ]:
res.to_csv(destData,
           sep='\t',
           )

In [ ]:
%matplotlib inline
valMax = res['pCH4Wtr'].max

fig = plt.figure()

# wether this or those
plt.scatter(res['long'], res['lat'], c='r', s=10, marker='s')
plt.plot(obris['long'], obris['lat'], 'b-')

fig.savefig('gis.png')

print(df)
print(res)